### 问题 1 具体解题思路和算法

**目标**: 设计抽样检测方案，以尽可能少的检测次数，在给定的信度下判断零配件次品率是否超过或不超过标称值。

**思路**:
这个问题实际上可以归结为统计学中的假设检验问题。我们通过抽样检测来判断样本的次品率，并利用置信区间或显著性水平来做出决策。

假设：
- **零假设** \( H_0 \): 次品率 \( p \leq 10\% \)（标称值）；
- **备择假设** \( H_1 \): 次品率 \( p > 10\% \)。

根据题目要求：
1. **拒收标准**：在 95% 的信度下（显著性水平 \( \alpha = 5\% \)）认定次品率超过标称值；
2. **接收标准**：在 90% 的信度下（显著性水平 \( \alpha = 10\% \)）认定次品率不超过标称值。

### 详细算法步骤

1. **抽样模型**:
   抽样检测可以看作是二项分布问题，假设从一批零配件中抽取 \( n \) 个样品，其中次品数量 \( X \) 服从二项分布：

   $$ X \sim \text{Bin}(n, p) $$

   其中 \( p \) 为真实的次品率，样本量为 \( n \)。

   根据大数定理，当 \( n \) 较大时，二项分布可以近似为正态分布：

   $$ \hat{p} \sim \mathcal{N}(p, \frac{p(1-p)}{n}) $$

   其中 \( \hat{p} \) 为样本次品率。

2. **确定样本量 \( n \)**:
   我们希望通过尽可能少的样本数来实现给定的信度。根据正态分布的性质，在给定显著性水平 \( \alpha \) 的情况下，临界值 \( z_{\alpha/2} \) 可以从标准正态表中查得。

   对于拒收标准（95% 的信度），我们使用 95% 置信区间，即：

   $$ z_{0.05/2} \approx 1.96 $$

   对于接收标准（90% 的信度），我们使用 90% 置信区间，即：

   $$ z_{0.10/2} \approx 1.645 $$

   对于二项分布，次品率的置信区间可以表示为：

   $$ \hat{p} \pm z_{\alpha/2} \cdot \sqrt{\frac{\hat{p}(1-\hat{p})}{n}} $$

   我们可以通过此公式计算需要的样本量。

3. **拒收规则**:
   如果在抽样过程中观测到的次品率 \( \hat{p} \) 超过 10% 的置信区间上限，即

   $$ \hat{p} > 10\% + z_{0.05/2} \cdot \sqrt{\frac{0.1 \times 0.9}{n}} $$

   则在 95% 的信度下拒收该批次零配件。

4. **接收规则**:
   如果观测到的次品率 \( \hat{p} \) 不超过 10% 的置信区间下限，即

   $$ \hat{p} \leq 10\% - z_{0.10/2} \cdot \sqrt{\frac{0.1 \times 0.9}{n}} $$

   则在 90% 的信度下接收该批次零配件。

### 算法示例

- **确定拒收的样本量 \( n \)**：
  我们希望在 95% 的信度下拒收零配件。假设我们想以 5% 的误差容忍度来估计次品率，那么 \( z_{0.05/2} = 1.96 \)。根据公式：

  $$ n = \left( \frac{z_{\alpha/2}}{\text{误差}}\right)^2 \cdot \hat{p}(1-\hat{p}) $$

  取次品率的估计值为 10%：

  $$ n = \left( \frac{1.96}{0.05} \right)^2 \cdot 0.1 \cdot 0.9 \approx 138 $$

  因此，需要抽样大约 138 个零配件来检测是否在 95% 的信度下可以拒收该批次。

- **确定接收的样本量 \( n \)**：
  类似的，对于 90% 的信度和 5% 的误差容忍度，使用 \( z_{0.10/2} = 1.645 \)：

  $$ n = \left( \frac{1.645}{0.05} \right)^2 \cdot 0.1 \cdot 0.9 \approx 98 $$

  因此，需要抽样大约 98 个零配件来判断是否在 90% 的信度下可以接收该批次。

### 结果分析

1. **拒收（95% 信度）**: 根据上述计算，抽样 138 个零配件，如果次品率的样本估计值 \( \hat{p} \) 大于上限，则拒收该批次。
2. **接收（90% 信度）**: 抽样 98 个零配件，如果次品率的样本估计值 \( \hat{p} \) 小于下限，则接收该批次。

通过这样的抽样方案，可以在合理的抽样成本下，保证企业在不同信度下对零配件进行接收与否的决策。

In [3]:
import math
from scipy.stats import norm


# 计算样本量的函数
def calculate_sample_size(p_hat, error_margin, confidence_level):
    # 获取正态分布临界值 z_score
    z_score = norm.ppf(1 - (1 - confidence_level) / 2)

    # 计算样本量
    n = (z_score**2 * p_hat * (1 - p_hat)) / (error_margin**2)

    return math.ceil(n)


# 样本检测的函数
def check_acceptance_rejection(
    sample_size, observed_defect_rate, p_nominal, confidence_level, accept=True
):
    z_score = norm.ppf(1 - (1 - confidence_level) / 2)
    # 计算临界值
    critical_value = p_nominal + z_score * math.sqrt(
        (p_nominal * (1 - p_nominal)) / sample_size
    )

    if accept:
        return observed_defect_rate <= critical_value
    else:
        return observed_defect_rate > critical_value


# 参数设定
p_nominal = 0.10  # 标称次品率10%
confidence_95 = 0.95  # 95% 信度
confidence_90 = 0.90  # 90% 信度
error_margin = 0.05  # 误差容忍度

# 计算拒收方案 (95% 信度)
sample_size_95 = calculate_sample_size(p_nominal, error_margin, confidence_95)
print(f"在95%的信度下拒收零配件所需的样本量: {sample_size_95}")

# 计算接收方案 (90% 信度)
sample_size_90 = calculate_sample_size(p_nominal, error_margin, confidence_90)
print(f"在90%的信度下接收零配件所需的样本量: {sample_size_90}")

# 假设我们从样本中得到了一个实际的次品率
observed_defect_rate = 0.12  # 实际观察到的次品率12%

# 检查是否拒收 (95% 信度)
is_reject_95 = check_acceptance_rejection(
    sample_size_95, observed_defect_rate, p_nominal, confidence_95, accept=False
)
print(f"在95%的信度下，是否拒收零配件: {'是' if is_reject_95 else '否'}")

# 检查是否接收 (90% 信度)
is_accept_90 = check_acceptance_rejection(
    sample_size_90, observed_defect_rate, p_nominal, confidence_90, accept=True
)
print(f"在90%的信度下，是否接收零配件: {'是' if is_accept_90 else '否'}")

在95%的信度下拒收零配件所需的样本量: 139
在90%的信度下接收零配件所需的样本量: 98
在95%的信度下，是否拒收零配件: 否
在90%的信度下，是否接收零配件: 是


# 使用序贯分析法（Sequential Analysis），如Wald的序贯概率比测试（SPRT），减少平均样本数量（ASN）

**序贯分析法**（Sequential Analysis）通过在样本的每一步进行统计检验来尽早做出决策，进而减少平均样本数量（ASN）。**Wald 的序贯概率比检验（SPRT）** 是这种方法的经典之一。相较于传统的固定样本量方法，SPRT能在保证相同的显著性水平和检验功效下，通常用更少的样本数完成检验。

### SPRT的基本原理

Wald 的序贯概率比检验的关键思想是：
1. 在每次抽取一个样本后，计算该样本带来的**似然比**，并根据该比率来判断是否接受、拒绝或者继续抽样。
2. 如果似然比超过上限，则拒收；如果低于下限，则接收；如果介于两者之间，则继续抽样。

### Wald's SPRT 设定

1. **假设**：
   - **零假设**：\( H_0 \)：次品率 \( p \leq p_0 \) （例如标称值 \( p_0 = 0.1 \)）。
   - **备择假设**：\( H_1 \)：次品率 \( p > p_1 \)（一个比 \( p_0 \) 大的值，比如 \( p_1 = 0.12 \)）。

2. **序贯检验准则**：
   - 每次抽取一个样本后，计算样本的似然比：
   
     $$
     \Lambda_n = \prod_{i=1}^{n} \frac{P(X_i | H_1)}{P(X_i | H_0)}
     $$

     其中 \( X_i \) 是第 \( i \) 个样本的结果，\( P(X_i | H_1) \) 和 \( P(X_i | H_0) \) 分别表示假设 \( H_1 \) 和 \( H_0 \) 下观测值的概率。

3. **决策边界**：
   - 当 \( \Lambda_n \geq A \) 时，拒绝 \( H_0 \)，接受 \( H_1 \)。
   - 当 \( \Lambda_n \leq B \) 时，接受 \( H_0 \)，拒绝 \( H_1 \)。
   - 当 \( B < \Lambda_n < A \) 时，继续抽样。

   上下限 \( A \) 和 \( B \) 由显著性水平 \( \alpha \) 和检验功效 \( \beta \) 确定：

   $$
   A = \frac{1-\beta}{\alpha}, \quad B = \frac{\beta}{1-\alpha}
   $$

### Python实现SPRT

以下代码实现了 Wald 的序贯概率比检验：


In [4]:
import numpy as np

# SPRT参数设定
p_0 = 0.10  # 零假设下的次品率
p_1 = 0.12  # 备择假设下的次品率
alpha = 0.05  # 显著性水平
beta = 0.10  # 允许的检验功效

# 计算A和B
A = (1 - beta) / alpha
B = beta / (1 - alpha)


# SPRT 检验函数
def SPRT(p_0, p_1, alpha, beta, observations):
    log_A = np.log(A)
    log_B = np.log(B)

    # 初始化 log(似然比)
    log_likelihood_ratio = 0

    for i, obs in enumerate(observations):
        # 计算当前样本的似然比
        log_likelihood_ratio += np.log(p_1 / p_0 if obs == 1 else (1 - p_1) / (1 - p_0))

        # 检查是否达到接受或拒绝边界
        if log_likelihood_ratio >= log_A:
            return "Reject H0 (次品率超过标称值)", i + 1
        elif log_likelihood_ratio <= log_B:
            return "Accept H0 (次品率不超过标称值)", i + 1

    return "Continue Sampling", len(observations)


# 模拟数据生成（次品率为12%）
np.random.seed(42)
observations = np.random.binomial(1, 0.12, 100)  # 生成100个样本，每个样本为0或1

# 进行SPRT检验
result, sample_used = SPRT(p_0, p_1, alpha, beta, observations)
print(f"结果: {result}，使用的样本数: {sample_used}")

结果: Continue Sampling，使用的样本数: 100



### 代码解释：
1. **SPRT参数**：
   - `p_0 = 0.10`：零假设下的次品率。
   - `p_1 = 0.12`：备择假设下的次品率。
   - `alpha = 0.05`：显著性水平，表示假阳性率。
   - `beta = 0.10`：检验功效，表示假阴性率。

2. **SPRT检验函数**：
   - 每次循环都会对当前样本计算**似然比**并累积到 `log_likelihood_ratio`。
   - 如果**似然比**超过上界 \( A \)，则拒绝零假设（即次品率超过标称值）。
   - 如果**似然比**低于下界 \( B \)，则接受零假设（即次品率不超过标称值）。
   - 如果似然比在上下界之间，则继续抽样。

3. **模拟数据**：
   - 使用 `np.random.binomial` 来生成一个包含次品率为12%的模拟样本序列（共100个样本）。
   - 调用SPRT函数来进行检验，输出是否接受或拒绝以及使用的样本数。

### 示例结果：
假设我们生成了100个次品率为12%的样本，序贯分析可能在第50个样本后就已经得出了结论，从而节省了50%的样本检测。

### 优点：
- 通过序贯分析法，在不损失检验效能的情况下，显著减少了平均样本量（ASN），从而降低了检测成本和时间。